In [1]:

from FCSRN import fcsrn
#from FCSRN_AugLoss import fcsrn

import tensorflow as tf
import cv2
from pathlib import Path
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import time


In [2]:
HEIGHT = 48; WIDTH = 160; CHANNELS = 3
IMAGE_SHAPE = (HEIGHT, WIDTH, CHANNELS)
EPOCHS = 100
BATCH_SIZE = 100
N_TRAIN = 4000
N_TEST = 1000

BATCHES_TRAIN = N_TRAIN / BATCH_SIZE
BATCHES_TEST = N_TEST / BATCH_SIZE

# load data + labels

In [3]:
# resize image and add padding on one dimensions to keep ratios
def resize_withPadding(image, targetWidth=WIDTH, targetHeight=HEIGHT):    
    inputWidth = image.shape[1]
    inputHeight = image.shape[0]
    # scale either width or height, depending on which scaling factor would be smaller
    scale_width = targetWidth / inputWidth
    scale_height = targetHeight / inputHeight
    
    if scale_width < scale_height:
        # scale width, pad height
        result = cv2.resize(image, dsize=(0,0), fx=scale_width, fy=scale_width)
        padding = targetHeight - result.shape[0]
        p_top = int(padding/2)
        p_bot = p_top if (padding%2) == 0 else p_top + 1
        assert padding >= 0 and (p_top + p_bot) == padding, "unexpected height-padding: %d"%(padding)
        result = cv2.copyMakeBorder(result, top=p_top, bottom=p_bot, left=0, right=0, borderType=cv2.BORDER_CONSTANT,value=0)
    else:
        # scale height, pad width
        result = cv2.resize(image, dsize=(0,0), fx=scale_height, fy=scale_height)
        padding = targetWidth - result.shape[1]
        p_left = int(padding/2)
        p_right = p_left if (padding%2) == 0 else p_left + 1
        assert padding >= 0 and (p_left + p_right) == padding, "unexpected width-padding: %d"%(padding)
        result = cv2.copyMakeBorder(result, top=0, bottom=0, left=p_left, right=p_right, borderType=cv2.BORDER_CONSTANT,value=0)                                    
    return result

# open and resize an image
def loadImage(imgPath, height=HEIGHT, width=WIDTH):
    image = cv2.imread(str(imgPath), cv2.IMREAD_COLOR)
    image = resize_withPadding(image, width, height)
    assert image.shape[0] == height and image.shape[1] == width, "resizing failed"
    return image

In [4]:
# load data
datasetPath = Path("C:/Users/andre/Desktop/m/datasets/SCUT-WMN DataSet")
trainfile = datasetPath / "difficult_samples_for_train.txt"
testfile = datasetPath / "difficult_samples_for_test.txt"

# load trainimages
df_train = pd.read_csv(trainfile, sep="[ ,]" ,header=None)
images_train = np.array([
    loadImage(datasetPath / row[0]) for row in df_train.values
]).astype("float32")
labels_train = np.array([
    row[1:] for row in df_train.values
]).astype("int")

# load testimages
df_test = pd.read_csv(testfile, sep="[ ,]" ,header=None)
images_test = np.array([
    loadImage(datasetPath / row[0]) for row in df_test.values
]).astype("float32")
labels_test = np.array([
    row[1:] for row in df_test.values
]).astype("int")

print(images_train.shape)
print(labels_train.shape)

C:\Users\andre\Miniconda3\envs\keras_gpu\lib\site-packages\ipykernel_launcher.py:7: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  import sys
C:\Users\andre\Miniconda3\envs\keras_gpu\lib\site-packages\ipykernel_launcher.py:16: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  app.launch_new_instance()


(4000, 48, 160, 3)
(4000, 5)


In [5]:
# make tf-datasets
train_X = tf.data.Dataset.from_tensor_slices(images_train[0:N_TRAIN])
train_y = tf.data.Dataset.from_tensor_slices(labels_train[0:N_TRAIN])

test_X = tf.data.Dataset.from_tensor_slices(images_test[0:N_TEST])
test_y = tf.data.Dataset.from_tensor_slices(labels_test[0:N_TEST])

train_X = train_X \
    .cache() \
    .batch(BATCH_SIZE)
    
train_y = train_y \
    .cache() \
    .batch(BATCH_SIZE)

test_X = test_X \
    .cache() \
    .batch(BATCH_SIZE)
    
test_y = test_y \
    .cache() \
    .batch(BATCH_SIZE)


print(train_X.element_spec)
print(train_y.element_spec)

TensorSpec(shape=(None, 48, 160, 3), dtype=tf.float32, name=None)
TensorSpec(shape=(None, 5), dtype=tf.int32, name=None)


# build model

In [6]:
model_savepath = Path("D:/m2/savedmodels/FCSRN_with_initializer")
model = fcsrn(IMAGE_SHAPE, checkpoint_path=model_savepath)

loaded checkpoint:  D:\m2\savedmodels\FCSRN_with_initializer\epoch-100


In [7]:
#model.model.summary()

In [8]:
#print("Training %d batches of size %d for %d epochs" % (BATCHES_TRAIN, BATCH_SIZE, EPOCHS) )
#model.train(train_X, train_y, EPOCHS, BATCH_SIZE)

In [9]:
total_characters = N_TEST * 5
lcr, ar, mismatches = model.accuracy(test_X, test_y, BATCH_SIZE, total_characters, N_TEST)
print("lcr: %f" % (lcr))
print("ar: %f" % (ar))

Instructions for updating:
Create a `tf.sparse.SparseTensor` and use `tf.sparse.to_dense` instead.
lcr: 0.856000
ar: 0.968600


In [10]:
"""print(mismatches[0][2])
sum_dist = 0
for row in mismatches:
    s = ""
    s += str(row[0].numpy())
    s += str(row[1].numpy())
    s += str(row[2].numpy())
    print(s)
    sum_dist += row[2]
print(sum_dist)"""

'print(mismatches[0][2])\nsum_dist = 0\nfor row in mismatches:\n    s = ""\n    s += str(row[0].numpy())\n    s += str(row[1].numpy())\n    s += str(row[2].numpy())\n    print(s)\n    sum_dist += row[2]\nprint(sum_dist)'